**数据集提取器😂**

Powered by LSA2010

In [20]:
pip install tqdm

In [26]:
import subprocess

subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True)

CompletedProcess(args='curl -fsSL https://ollama.com/install.sh | sh', returncode=0)

In [28]:
import subprocess, time

# 启动 ollama-serve
ollama_proc = subprocess.Popen("ollama serve", shell=True)

time.sleep(5)
print("Ollama_runing")

Ollama_runing


In [9]:
!ollama pull gpt-oss:20b

In [ ]:
import requests
import json
import random
from tqdm import tqdm

def generate_data():
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gpt-oss:20b",
        "prompt": """每个样本应包含一个英文单词和需要计数的字母。
将结果输出为JSON格式,每个样本包含"question"和"answer"两个字段。
"question"字段应包含询问某个单词中某个字母出现次数的问题,而"answer"字段应包含逐个字母分析的过程和正确答案。
示例输出格式:
[
{
"question": "strawberry这个单词中有几个字母r？",
"answer": "让我逐个字母来数一下strawberry中r的个数：s-t-r-a-w-b-e-r-r-y。第3个位置是r，第8个位置是r，第9个位置也是r。因此strawberry中一共有3个字母r。"
},
{
"question": "单词banana中字母a出现了多少次？",
"answer": "我来逐个检查banana中每个字母：b-a-n-a-n-a。第2个位置是a，第4个位置是a，第6个位置是a。所以banana中字母a出现了3次。"
}
]
请确保:
1.单词的长度和复杂程度应该多样化，包括短单词和长单词。
2.有些单词中目标字母可能出现多次，有些可能不出现，以增加难度。
3.问题的表述应该有所变化，如"有几个字母"、"出现了多少次"、"包含多少个"等。
4答案应该包含逐个字母分析的详细过程，明确指出每个目标字母的位置。
5.选择不同的目标字母进行计数，如a、e、i、o、u、r、s、t等。
6.包含一些容易混淆的情况，比如大小写、相似字母等。
请生成符合上述要求的1个训练样本。"""
    }

    response = requests.post(url, json=payload, stream=True)
    result = ""

    for line in response.iter_lines():
        if line:
            data = json.loads(line.decode("utf-8"))
            if "response" in data:
                result += data["response"]
            if data.get("done", False):
                break

    return result

def save_to_file(data, filename="data_coder.json", silent=False):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            existing_data = json.load(f)
    except FileNotFoundError:
        existing_data = []

    try:
        new_data = json.loads(data)
        if isinstance(new_data, list):
            existing_data.extend(new_data)
        else:
            existing_data.append(new_data)
    except json.JSONDecodeError:
        if not silent:
            print("Failed to parse response as JSON")
        return

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, indent=2, ensure_ascii=False)

    if not silent:
        print(f"Data saved to {filename}")

def generate_multiple_samples(count=100):
    for i in tqdm(range(count), desc="Generating samples", unit="sample"):
        try:
            result = generate_data()
            save_to_file(result, silent=True)
        except Exception as e:
            tqdm.write(f"Error generating sample {i+1}: {e}")
            continue
    print(f"Successfully generated {count} samples and saved to data_coder.json")

if __name__ == "__main__":
    generate_multiple_samples(100)


Generating samples:   1%|          | 1/100 [01:39<2:44:23, 99.63s/sample]